<a href="https://colab.research.google.com/github/Explorer9/INFO5731_FALL2020/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:

import pandas as pd
import tarfile
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import random 

In [70]:
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 317.85 KiB | 957.00 KiB/s, done.
Resolving deltas: 100% (185/185), done.


In [71]:
!git clone https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

Cloning into 'uncased_L-12_H-768_A-12.zip'...
fatal: unable to access 'https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip/': The requested URL returned error: 403


In [35]:

# Write your code here
import pandas as pd
import requests
from bs4 import BeautifulSoup
def insert_into_array(values,ref_array):
  for i in values:
    ref_array.append(i.text)
URL=["https://www.imdb.com/title/tt7294534/reviews/_ajax?ref_=undefined&paginationKey=g4wp7djfquydczye7wthxnztqltmwcbhzfmxvlnomwklyczuf43o6ssypazfjmjddv4k4rk5izrbr2k7kbghholct5fuigy","https://www.imdb.com/title/tt7294534/reviews/_ajax?ref_=undefined&paginationKey=g4wp7dbfqi3tizqg66uhhnzrrdt4ucbyy4hhzo5ziwr26fbyhvrl4ty4oqyvtprodfr5dtt3nlozdkpd6sssetvth324k3g7","https://www.imdb.com/title/tt7294534/reviews/_ajax?ref_=undefined&paginationKey=g4wp7dbqsr3ximsatx5dz2dezxzyyrr72intd6pnaljzoplumbhlqx7ok6yfifxz3a76iui","https://www.imdb.com/title/tt7294534/reviews/_ajax?ref_=undefined&paginationKey=g4wp7dbqsr3ximsatx5dz2dezxzyyrr73mptr4hfapjzpeaaaj5kx7i75r5y6odr55p7f2q"]
text=[]
rating=[]
review_title=[]
for i in URL:
  page=requests.get(i,headers={'User-Agent':'Chrome/85.0.4183.121'})
  soup = BeautifulSoup(page.content, 'html.parser')
  insert_into_array(soup.find_all('div', class_='text show-more__control'),text)
  insert_into_array(soup.find_all('a',class_='title'),review_title)
  insert_into_array(soup.find_all(name='span',class_='rating-other-user-rating'),rating)
imdb_df=pd.DataFrame({'comment':text,'review_title':review_title})
imdb_df.to_csv("imbd")
print(imdb_df[:6])

                                             comment                                       review_title
0  I can't understand what's so path breaking, in...                                 A wannabe Devdas\n
1  I have personally known people like Arjun Redd...   Arjun Reddy is a person who sees world in bla...
2  Best acting , mind blowing background scores d...    One of the realistic movie in Telugu industry\n
3  If "Rashmika Mandanna" was Preethi\nIt would b...                                               IF\n
4  Arjun Reddy is a story in purest. Its fresh, l...                   A milestone in romantic genre.\n
5  The movie displays plain and uncensored drug a...                                          Unique.\n


In [36]:
train_df = imdb_df['comment'][:50].to_frame()
train_df['sentiment']=''
for i in range(len(train_df)):
  train_df['sentiment'][i]=random.choice([0,1]) 
train_df.columns = range(train_df.shape[1])
train_df.head()

,0,1
0,"I can't understand what's so path breaking, in...",1
1,I have personally known people like Arjun Redd...,1
2,"Best acting , mind blowing background scores d...",1
3,"If ""Rashmika Mandanna"" was Preethi\nIt would b...",1
4,"Arjun Reddy is a story in purest. Its fresh, l...",1


In [37]:
test_df = imdb_df['comment'][50:80].to_frame()
test_df['sentiment']=''
for i in range(len(test_df)):
  test_df['sentiment'][i]=random.choice([0,1]) 
test_df.columns = range(test_df.shape[1])
train_df.head()


,0,1
0,"I can't understand what's so path breaking, in...",1
1,I have personally known people like Arjun Redd...,1
2,"Best acting , mind blowing background scores d...",1
3,"If ""Rashmika Mandanna"" was Preethi\nIt would b...",1
4,"Arjun Reddy is a story in purest. Its fresh, l...",1


In [65]:
a={'id':range(len(train_df)),'label':train_df[1],'alpha':['a']*train_df.shape[1],'text': train_df[0].replace(r'\n', ' ', regex=True)}
df_bert = pd.DataFrame.from_dict(a, orient='index')
df_bert = df_bert.transpose()

# Splitting training data file into *train* and *dev*
df_bert_train, df_bert_dev = train_test_split(df_bert, test_size=0.01)

df_bert_train.head()

,id,label,alpha,text
10,10,1,None,An absolute masterpiece. Not to be missed. I'v...
40,40,1,None,What a experience omg was totally blown away f...
2,2,1,None,"Best acting , mind blowing background scores d..."
15,15,1,None,People who never had a relationship or have a ...
23,23,0,None,"The best there was, the best there is and prob..."


In [67]:
df_bert_test = pd.DataFrame({
    'id':range(len(test_df)),
    'text': test_df[0].replace(r'\n', ' ', regex=True)
})

df_bert_test.head()

,id,text
50,0,All the characters in this movie are so cute. ...
51,1,The director directed the movie in the way tha...
52,2,Boyfriend ho toh Arjun Reddy jaisa ho varna na...
53,3,"Undoubtedly, Arjun Reddy is a Winner.. The way..."
54,4,This is my first review and I am completely to...


In [69]:
# Saving dataframes to .tsv format as required by BERT
df_bert_train.to_csv('/content/data/train.tsv', sep='\t', index=False, header=False)
df_bert_dev.to_csv('/content/data/dev.tsv', sep='\t', index=False, header=False)
df_bert_test.to_csv('/content/data/test.tsv', sep='\t', index=False, header=False)

In [ ]:
!python /content/bert/run_classifier.py 
--task_name=cola
--do_train=true 
--do_eval=true 
--do_predict=true 
--data_dir="/content/data"
--vocab_file="/content/drive/My Drive/uncased_L-12_H-768_A-12/vocab.txt"
--bert_config_file="/content/drive/My Drive/uncased_L-12_H-768_A-12/bert_config.json"
--init_checkpoint="/content/drive/My Drive/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001"
--max_seq_length=128 
--train_batch_size=32 
--learning_rate=2e-5 
--num_train_epochs=3.0 
--output_dir="/bert_output/" 
--do_lower_case=False